# PROYECTO - Inferencia de Ingresos

Este notebook implementa el pipeline completo requerido por la rúbrica:
- Carga y concatenación de nóminas públicas (CSV)
- Limpieza y preprocesamiento
- EDA (estadísticas y visualizaciones)
- Entrenamiento de 10 modelos de regresión exigidos
- Evaluación y comparación de métricas
- Función para predecir desde un nuevo archivo CSV

**Instrucciones:** Los archivos CSV en la carpeta `./data/` antes de ejecutar. 

Las nomínas son de los siguientes hospitales:
| Acrónimo   | Hospital                                           |URL
| ---------- | -------------------------------------------------- |-----
| **HDPB**   | Hospital Docente Padre Billini                     |https://datos.gob.do/dataset/nomina-de-empleados-hdpb-2018
| **HDSSD**  | Hospital Docente San Salvador del Distrito         |https://datos.gob.do/dataset/hospital-docente-semma
| **HDUDDC** | Hospital Docente Universitario Dr. Darío Contreras |https://datos.gob.do/dataset/h-d-c
| **HGDVC**  | Hospital General Docente de Villa Consuelo         |https://datos.gob.do/dataset/nomina_de_empleados


In [25]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Modelos de regresión requeridos
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

print('Librerías importadas correctamente')

Librerías importadas correctamente


In [26]:
# Cargar todos los CSV en ./data/
data_folder = Path('./data/csv/')
data_folder.mkdir(exist_ok=True)
csv_files = sorted(glob.glob(str(data_folder / '*.csv')))

print(f'Archivos CSV detectados en ./data/: {len(csv_files)}')
for f in csv_files[:10]:
    print(' -', f)

def generate_demo_dataset(path, n=6000, random_state=42):
    """Genera un dataset sintético de demostración guardado como CSV."""
    rng = np.random.RandomState(random_state)
    cargos = ['Técnico', 'Analista', 'Administrador', 'Director', 'Asistente']
    instituciones = ['Ministerio A', 'Ministerio B', 'Ayuntamiento X', 'Universidad Y', 'Hospital Z']
    municipios = ['Santo Domingo', 'Santiago', 'La Vega', 'San Pedro', 'Puerto Plata']
    genero = ['M', 'F']
    tipo_nom = ['Nombrado', 'Contratado', 'Por Servicio']

    df = pd.DataFrame({
        'institucion': rng.choice(instituciones, size=n),
        'dependencia': rng.choice(['Dept1','Dept2','Dept3','Dept4'], size=n),
        'cargo': rng.choice(cargos, size=n),
        'nivel': rng.randint(1,10,size=n),
        'tipo_nombramiento': rng.choice(tipo_nom, size=n),
        'fecha_ingreso': pd.to_datetime('2010-01-01') + pd.to_timedelta(rng.randint(0,4000,size=n), unit='D'),
        'municipio': rng.choice(municipios, size=n),
        'sueldo_bruto': np.round(rng.normal(35000, 12000, size=n).clip(8000, 120000), 2),
        'incentivos': np.round(np.abs(rng.normal(2000,1500,size=n)), 2),
        'sueldo_neto': None,
        'genero': rng.choice(genero, size=n)
    })
    # define sueldo_neto as bruto + incentivos + small noise
    df['sueldo_neto'] = np.round(df['sueldo_bruto'] + df['incentivos'] - rng.normal(1500,500,size=n),2)
    path.mkdir(parents=True, exist_ok=True)
    out = path / 'demo_nomina.csv'
    df.to_csv(out, index=False)
    print('Dataset de demostración creado en', out)
    return str(out)

if len(csv_files) == 0:
    print('No se encontraron CSV en ./data/. Generando un dataset de demostración...')
    demo = generate_demo_dataset(data_folder, n=6000)
    csv_files = [demo]

# Cargar y concatenar
dfs = []
for f in csv_files:
    try:
        df = pd.read_csv(f)
        df['_source_file'] = os.path.basename(f)
        dfs.append(df)
    except Exception as e:
        print('Error cargando', f, e)

raw = pd.concat(dfs, ignore_index=True)
print('\nDataset concatenado shape:', raw.shape)
raw.head().T

Archivos CSV detectados en ./data/: 4
 - data\csv\HDPB-Nomina-2024.csv
 - data\csv\HDSSD-Nomina-2025.csv
 - data\csv\HDUDDC-Nomina-2025.csv
 - data\csv\HGDVC-Nomina-2025.csv

Dataset concatenado shape: (85008, 32)


,0,1,2,3,4
NOMBRE,YADENYS DEL CARMEN,ERISMERY MASIEL,SANTA RAYSA,MARIA ALTAGRACIA,FLOR KATIUSKA
APELLIDO,TORIBIO,QUEZADA ALMANZAR,HEREDIA PEREZ,PAREDES,ORTEGA
DEPARTAMENTO,DIRECCION GENERAL,DIRECCION GENERAL,SUB- DIRECCION MEDICA,SUB- DIRECCION MEDICA,SUB- DIRECCION MEDICA
CARGO QUE DESEMPEÑA,ASISTENTE EJECUTIVA,SECRETARIA DIRECCION,JEFA DE MEDICINA INTERNA Y ESPECIALIDADES,ASISTENTE ADM. SUB-DIRECCION,GERENTE OPERATIVO
SUELDO BASE,20000,15000,60000,15000,40000
COMPLETIVO A SUELDO,0,0,0,0,0
TOTAL DE SUELDO,20000,15000,60000,15000,40000
TIPO DE EMPLEADO,CONTRATADO INTERNO,CONTRATADO INTERNO,CONTRATADO INTERNO,CONTRATADO INTERNO,CONTRATADO INTERNO
MES,ENERO,ENERO,ENERO,ENERO,ENERO
AÑO,2019,2019,2019,2019,2019


In [27]:
# Vista general y limpieza inicial
def overview(df, n=5):
    print('Shape:', df.shape)
    print('\nTipos de datos:')
    print(df.dtypes)
    print('\nNulos por columna:')
    print(df.isnull().sum().sort_values(ascending=False).head(20))
    display(df.head(n))

overview(raw)

Shape: (85008, 32)

Tipos de datos:
NOMBRE                  object
APELLIDO                object
DEPARTAMENTO            object
CARGO QUE DESEMPEÑA     object
SUELDO BASE             object
COMPLETIVO A SUELDO     object
TOTAL DE SUELDO         object
TIPO DE EMPLEADO        object
MES                     object
AÑO                     object
_source_file            object
Nombre                  object
Genero                  object
Departamento            object
Posicion                object
Estatus                 object
Sueldo Bruto            object
Otros Ingresos         float64
Total Ingresos         float64
ISR                    float64
Seguro Medico          float64
Seguro Vejez           float64
Otros Descuentos       float64
Sueldo Neto            float64
Mes                     object
Año                    float64
Nombres                 object
Funcion                 object
Sueldo                 float64
Mes                     object
Apellido                object
Fun

,NOMBRE,APELLIDO,DEPARTAMENTO,CARGO QUE DESEMPEÑA,SUELDO BASE,COMPLETIVO A SUELDO,TOTAL DE SUELDO,TIPO DE EMPLEADO,MES,AÑO,_source_file,Nombre,Genero,Departamento,Posicion,Estatus,Sueldo Bruto,Otros Ingresos,Total Ingresos,ISR,Seguro Medico,Seguro Vejez,Otros Descuentos,Sueldo Neto,Mes,Año,Nombres,Funcion,Sueldo,Mes,Apellido,Función
0,YADENYS DEL CARMEN,TORIBIO,DIRECCION GENERAL,ASISTENTE EJECUTIVA,20000,0,20000,CONTRATADO INTERNO,ENERO,2019,HDPB-Nomina-2024.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ERISMERY MASIEL,QUEZADA ALMANZAR,DIRECCION GENERAL,SECRETARIA DIRECCION,15000,0,15000,CONTRATADO INTERNO,ENERO,2019,HDPB-Nomina-2024.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SANTA RAYSA,HEREDIA PEREZ,SUB- DIRECCION MEDICA,JEFA DE MEDICINA INTERNA Y ESPECIALIDADES,60000,0,60000,CONTRATADO INTERNO,ENERO,2019,HDPB-Nomina-2024.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MARIA ALTAGRACIA,PAREDES,SUB- DIRECCION MEDICA,ASISTENTE ADM. SUB-DIRECCION,15000,0,15000,CONTRATADO INTERNO,ENERO,2019,HDPB-Nomina-2024.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FLOR KATIUSKA,ORTEGA,SUB- DIRECCION MEDICA,GERENTE OPERATIVO,40000,0,40000,CONTRATADO INTERNO,ENERO,2019,HDPB-Nomina-2024.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Normalización de nombres de columnas
raw.columns = [c.strip().lower().replace(' ', '_') for c in raw.columns]
raw.rename(columns=lambda x: x.replace('#','').replace('/','_'), inplace=True)
print('Columnas tras normalizar:', list(raw.columns))

Columnas tras normalizar: ['nombre', 'apellido', 'departamento', 'cargo_que_desempeña', 'sueldo_base', 'completivo_a_sueldo', 'total_de_sueldo', 'tipo_de_empleado', 'mes', 'año', '_source_file', 'nombre', 'genero', 'departamento', 'posicion', 'estatus', 'sueldo_bruto', 'otros_ingresos', 'total_ingresos', 'isr', 'seguro_medico', 'seguro_vejez', 'otros_descuentos', 'sueldo_neto', 'mes', 'año', 'nombres', 'funcion', 'sueldo', 'mes', 'apellido', 'función']


In [29]:
# Asegurar tipos: intentar convertir fechas y montos
for c in raw.columns:
    if 'fecha' in c or 'date' in c:
        try:
            raw[c] = pd.to_datetime(raw[c], errors='coerce')
        except:
            pass

# detectar columnas numéricas que contienen comas o signos y limpiarlas
possible_num = ['sueldo_bruto','sueldo_neto','incentivos']
for c in raw.columns:
    if c in possible_num and raw[c].dtype == object:
        raw[c] = raw[c].astype(str).str.replace(',', '').str.replace('\$', '').replace('', np.nan)
        raw[c] = pd.to_numeric(raw[c], errors='coerce')

overview(raw)

Shape: (85008, 32)

Tipos de datos:
nombre                  object
apellido                object
departamento            object
cargo_que_desempeña     object
sueldo_base             object
completivo_a_sueldo     object
total_de_sueldo         object
tipo_de_empleado        object
mes                     object
año                     object
_source_file            object
nombre                  object
genero                  object
departamento            object
posicion                object
estatus                 object
sueldo_bruto           float64
otros_ingresos         float64
total_ingresos         float64
isr                    float64
seguro_medico          float64
seguro_vejez           float64
otros_descuentos       float64
sueldo_neto            float64
mes                     object
año                    float64
nombres                 object
funcion                 object
sueldo                 float64
mes                     object
apellido                object
fun

,nombre,apellido,departamento,cargo_que_desempeña,sueldo_base,completivo_a_sueldo,total_de_sueldo,tipo_de_empleado,mes,año,_source_file,nombre,genero,departamento,posicion,estatus,sueldo_bruto,otros_ingresos,total_ingresos,isr,seguro_medico,seguro_vejez,otros_descuentos,sueldo_neto,mes,año,nombres,funcion,sueldo,mes,apellido,función
0,YADENYS DEL CARMEN,TORIBIO,DIRECCION GENERAL,ASISTENTE EJECUTIVA,20000,0,20000,CONTRATADO INTERNO,ENERO,2019,HDPB-Nomina-2024.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ERISMERY MASIEL,QUEZADA ALMANZAR,DIRECCION GENERAL,SECRETARIA DIRECCION,15000,0,15000,CONTRATADO INTERNO,ENERO,2019,HDPB-Nomina-2024.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SANTA RAYSA,HEREDIA PEREZ,SUB- DIRECCION MEDICA,JEFA DE MEDICINA INTERNA Y ESPECIALIDADES,60000,0,60000,CONTRATADO INTERNO,ENERO,2019,HDPB-Nomina-2024.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MARIA ALTAGRACIA,PAREDES,SUB- DIRECCION MEDICA,ASISTENTE ADM. SUB-DIRECCION,15000,0,15000,CONTRATADO INTERNO,ENERO,2019,HDPB-Nomina-2024.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FLOR KATIUSKA,ORTEGA,SUB- DIRECCION MEDICA,GERENTE OPERATIVO,40000,0,40000,CONTRATADO INTERNO,ENERO,2019,HDPB-Nomina-2024.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# Copia del dataset original
df = raw.copy()

# Normalizar nombres: quitar espacios, convertir a minúsculas
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Crear columnas estándar
df['sueldo_bruto'] = np.nan
df['incentivos'] = np.nan
df['sueldo_neto'] = np.nan

# Mapeo según los nombres posibles
if 'sueldo_base' in df.columns:
    df['sueldo_bruto'] = df['sueldo_base']

if 'completivo_a_sueldo' in df.columns:
    df['incentivos'] = df['completivo_a_sueldo']

if 'total_de_sueldo' in df.columns:
    df['sueldo_neto'] = df['total_de_sueldo']

if 'sueldo_bruto' in df.columns:
    df['sueldo_bruto'] = df['sueldo_bruto']

if 'otros_ingresos' in df.columns:
    df['incentivos'] = df['otros_ingresos']

if 'total_ingresos' in df.columns:
    df['sueldo_bruto'] = df['total_ingresos']

if 'sueldo' in df.columns:  # HDUDDC
    df['sueldo_bruto'] = df['sueldo']

# HGDVC sueldos con coma
if 'sueldo_bruto' in df.columns:
    df['sueldo_bruto'] = (
        df['sueldo_bruto']
        .astype(str)
        .str.replace(',', '')
        .str.replace(' ', '')
        .str.replace('.00', '', regex=False)
        .astype(float)
        .fillna(df['sueldo_bruto'])
    )

# Si no hay sueldo_neto, usar sueldo_bruto - descuentos si existen
if df['sueldo_neto'].isna().all():
    posibles_desc = ['isr', 'seguro_medico', 'seguro_vejez', 'otros_descuentos']
    if all(c in df.columns for c in posibles_desc):
        df['sueldo_neto'] = (
            df['sueldo_bruto']
            - df['isr'] 
            - df['seguro_medico']
            - df['seguro_vejez']
            - df['otros_descuentos']
        )
    else:
        df['sueldo_neto'] = df['sueldo_bruto']
display(df[['sueldo_bruto','incentivos','sueldo_neto']].describe())

,sueldo_bruto,incentivos
count,5886.000000,2000.000000
mean,13307.478253,1598.882105
std,6854.034762,3110.507654
min,1000.000000,0.000000
25%,10000.000000,0.000000
50%,10000.000000,0.000000
75%,13530.000000,2000.000000
max,45000.000000,28827.950000


In [31]:
# Reporte de calidad de datos
def quality_report(df):
    total_cells = df.size
    missing = df.isnull().sum().sum()
    pct_missing = missing / total_cells
    print(f'Total celdas: {total_cells:,}')
    print(f'Celdas con nulos: {missing:,} ({pct_missing:.2%})')

quality_report(df)

Total celdas: 2,805,264
Celdas con nulos: 1,944,823 (69.33%)


In [32]:
# Verificar que tengamos al menos 6 características y 1 etiqueta numérica
# Seleccionaremos 'sueldo_neto' como etiqueta por defecto
target = 'sueldo_neto'
features = [c for c in df.columns if c != target and not c.startswith('_source')]
print('Total features candidate:', len(features))
print('Lista (muestra):', features[:20])

# Si la etiqueta tiene muchos nulos, intentar usar sueldo_bruto
if df[target].isnull().mean() > 0.5:
    if 'sueldo_bruto' in df.columns:
        print('sueldo_neto tiene muchos nulos, usaremos sueldo_bruto como target')
        target = 'sueldo_bruto'

print('Etiqueta objetivo final:', target)

Total features candidate: 31
Lista (muestra): ['nombre', 'apellido', 'departamento', 'cargo_que_desempeña', 'sueldo_base', 'completivo_a_sueldo', 'total_de_sueldo', 'tipo_de_empleado', 'mes', 'año', 'nombre', 'genero', 'departamento', 'posicion', 'estatus', 'sueldo_bruto', 'otros_ingresos', 'total_ingresos', 'isr', 'seguro_medico']
sueldo_neto tiene muchos nulos, usaremos sueldo_bruto como target
Etiqueta objetivo final: sueldo_bruto


In [33]:
# Preparar dataset final para modelado
model_df = df.copy()

# Seleccionar columnas útiles: numéricas simples + algunas categóricas
numeric_cols = ['nivel','años_en_institucion','incentivos']
numeric_cols = [c for c in numeric_cols if c in model_df.columns]
categorical_cols = ['cargo','institucion','tipo_nombramiento','municipio','genero']
categorical_cols = [c for c in categorical_cols if c in model_df.columns]

print('Numeric cols:', numeric_cols)
print('Categorical cols:', categorical_cols)

# Rellenar nulos
model_df[numeric_cols] = model_df[numeric_cols].fillna(model_df[numeric_cols].median())
model_df[categorical_cols] = model_df[categorical_cols].fillna('desconocido')

# Drop rows where target is null
model_df = model_df[model_df[target].notnull()].copy()
print('Shape tras filtrar target nulo:', model_df.shape)

X = model_df[numeric_cols + categorical_cols]
y = model_df[target].astype(float)

# Simple train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

Numeric cols: ['incentivos']
Categorical cols: ['genero']
Shape tras filtrar target nulo: (5886, 33)
Train shape: (4708, 2) Test shape: (1178, 2)


In [34]:
# Construir preprocess pipeline
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
], remainder='drop')

In [35]:
# Definir modelos requeridos
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'BayesianRidge': BayesianRidge(),
    'Lasso': Lasso(max_iter=10000),
    'KNN': KNeighborsRegressor(),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'SVR': SVR(),
    'MLP': MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42)
}

results = {}
import time
for name, model in models.items():
    print('\nEntrenando', name)
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', model)])
    t0 = time.time()
    pipe.fit(X_train, y_train)
    t1 = time.time()
    preds = pipe.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    mse = mean_squared_error(y_test, preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, preds)
    results[name] = {'model': pipe, 'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2, 'train_time_s': t1-t0}
    print(f'{name} Listo. MAE={mae:.2f} RMSE={rmse:.2f} R2={r2:.4f} (train_time {t1-t0:.1f}s)')


Entrenando LinearRegression
LinearRegression Listo. MAE=4409.19 RMSE=6864.39 R2=-0.0003 (train_time 0.1s)

Entrenando Ridge
Ridge Listo. MAE=4409.19 RMSE=6864.39 R2=-0.0003 (train_time 0.0s)

Entrenando BayesianRidge
BayesianRidge Listo. MAE=4409.19 RMSE=6864.39 R2=-0.0003 (train_time 0.0s)

Entrenando Lasso
Lasso Listo. MAE=4409.19 RMSE=6864.39 R2=-0.0003 (train_time 0.0s)

Entrenando KNN
KNN Listo. MAE=4702.68 RMSE=6879.47 R2=-0.0047 (train_time 0.0s)

Entrenando DecisionTree
DecisionTree Listo. MAE=4409.19 RMSE=6864.39 R2=-0.0003 (train_time 0.1s)

Entrenando RandomForest
RandomForest Listo. MAE=4414.53 RMSE=6864.19 R2=-0.0003 (train_time 0.4s)

Entrenando SVR
SVR Listo. MAE=3578.31 RMSE=7662.68 R2=-0.2465 (train_time 1.9s)

Entrenando MLP
MLP Listo. MAE=3962.82 RMSE=6983.99 R2=-0.0355 (train_time 21.4s)

Entrenando AdaBoost
AdaBoost Listo. MAE=6244.60 RMSE=7410.70 R2=-0.1659 (train_time 0.1s)


In [36]:
# Resumen comparativo
res_df = pd.DataFrame([
    {'model':k, **{metric: v for metric,v in results[k].items() if metric!='model'}}
    for k in results
]).set_index('model').sort_values('rmse')

display(res_df)
# Guardar tabla de resultados
res_df.to_csv('modeling_results.csv', index=True)
print('Tabla de resultados guardada en modeling_results.csv')

,mae,mse,rmse,r2,train_time_s
model,,,,,
RandomForest,4414.528011,4.711709e+07,6864.188799,-0.000274,0.448843
LinearRegression,4409.185013,4.711991e+07,6864.394077,-0.000334,0.137550
Ridge,4409.185013,4.711991e+07,6864.394077,-0.000334,0.031975
BayesianRidge,4409.185013,4.711991e+07,6864.394077,-0.000334,0.017201
Lasso,4409.185013,4.711991e+07,6864.394077,-0.000334,0.018928
DecisionTree,4409.185013,4.711991e+07,6864.394077,-0.000334,0.062480
KNN,4702.676570,4.732713e+07,6879.471489,-0.004733,0.015621
MLP,3962.821850,4.877606e+07,6983.985984,-0.035493,21.422021
AdaBoost,6244.600551,5.491847e+07,7410.699408,-0.165894,0.099936


Tabla de resultados guardada en modeling_results.csv


In [37]:
# Seleccionar mejor modelo por RMSE y guardarlo
best_name = res_df.index[0]
best_model = results[best_name]['model']
print('Mejor modelo:', best_name)

models_dir = Path('./models')
models_dir.mkdir(exist_ok=True)
joblib.dump(best_model, models_dir / f'best_model_{best_name}.joblib')
print('Modelo guardado en', models_dir / f'best_model_{best_name}.joblib')

Mejor modelo: RandomForest
Modelo guardado en models\best_model_RandomForest.joblib


In [38]:
# Importancia de características (si aplica)
if best_name in ['RandomForest','DecisionTree']:
    # obtener nombres de columnas después del preprocessing
    ohe = best_model.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
    ohe_cols = list(ohe.get_feature_names_out(categorical_cols))
    final_cols = numeric_cols + ohe_cols
    importances = best_model.named_steps['regressor'].feature_importances_
    fi = pd.Series(importances, index=final_cols).sort_values(ascending=False)
    display(fi.head(30))
    fi.to_csv('feature_importances.csv')
    print('Importancias guardadas en feature_importances.csv')
else:
    print('El mejor modelo no provee importancias de features directamente.')

incentivos            0.0
genero_desconocido    0.0
dtype: float64

Importancias guardadas en feature_importances.csv


In [39]:
# Función para predecir desde un CSV nuevo
def predict_from_csv(csv_path, model_path=None):
    """Carga un CSV, aplica el mismo preprocesamiento y devuelve predicciones.
    Si model_path es None se usará el best_model actual en memoria.
    Devuelve un DataFrame con las columnas originales + prediction.
    """
    if model_path is None:
        model = best_model
    else:
        model = joblib.load(model_path)
    to_pred = pd.read_csv(csv_path)
    # normalizar nombres de columnas como en el notebook
    to_pred.columns = [c.strip().lower().replace(' ', '_') for c in to_pred.columns]
    # asegurarse que las columnas necesarias existan
    for c in numeric_cols:
        if c not in to_pred.columns:
            to_pred[c] = np.nan
    for c in categorical_cols:
        if c not in to_pred.columns:
            to_pred[c] = 'desconocido'
    # rellenar nulos
    to_pred[numeric_cols] = to_pred[numeric_cols].fillna(0)
    to_pred[categorical_cols] = to_pred[categorical_cols].fillna('desconocido')
    X_new = to_pred[numeric_cols + categorical_cols]
    preds = model.predict(X_new)
    to_pred['predicted_' + target] = preds
    return to_pred

# Ejemplo de uso (si existe archivo demo)
demo_csv = csv_files[0]
print('Probando predict_from_csv con:', demo_csv)
pred_df = predict_from_csv(demo_csv)
display(pred_df.head())

Probando predict_from_csv con: data\csv\HDPB-Nomina-2024.csv


,nombre,apellido,departamento,cargo_que_desempeña,sueldo_base,completivo_a_sueldo,total_de_sueldo,tipo_de_empleado,mes,año,incentivos,genero,predicted_sueldo_bruto
0,YADENYS DEL CARMEN,TORIBIO,DIRECCION GENERAL,ASISTENTE EJECUTIVA,20000,0,20000,CONTRATADO INTERNO,ENERO,2019,0.0,desconocido,13294.159169
1,ERISMERY MASIEL,QUEZADA ALMANZAR,DIRECCION GENERAL,SECRETARIA DIRECCION,15000,0,15000,CONTRATADO INTERNO,ENERO,2019,0.0,desconocido,13294.159169
2,SANTA RAYSA,HEREDIA PEREZ,SUB- DIRECCION MEDICA,JEFA DE MEDICINA INTERNA Y ESPECIALIDADES,60000,0,60000,CONTRATADO INTERNO,ENERO,2019,0.0,desconocido,13294.159169
3,MARIA ALTAGRACIA,PAREDES,SUB- DIRECCION MEDICA,ASISTENTE ADM. SUB-DIRECCION,15000,0,15000,CONTRATADO INTERNO,ENERO,2019,0.0,desconocido,13294.159169
4,FLOR KATIUSKA,ORTEGA,SUB- DIRECCION MEDICA,GERENTE OPERATIVO,40000,0,40000,CONTRATADO INTERNO,ENERO,2019,0.0,desconocido,13294.159169
